In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


from functions import main_cleaning

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

url = "https://www.sharkattackfile.net/spreadsheets/GSAF5.xls"

df_sa = pd.read_excel(url)


In [ ]:
df_sa.head()

In [ ]:
valid_species = {
    'Tiger shark', 'White shark', 'Bull shark', 'Hammerhead shark', 'Great white shark', 
    'Mako shark', 'Blacktip shark', 'Reef shark', 'Nurse shark', 'Whale shark', 'Tiger shark'
}

df_sa = main_cleaning(df_sa, valid_species)


In [ ]:
df_sa.head()

In [ ]:
df_sa = df_sa[df_sa["sex"] != "Lli"]
pivot_table = df_sa.pivot_table(index=["sex", "fatal"], columns="country", aggfunc="count")
pivot_table = pivot_table.fillna(0)
pivot_table

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Suponiendo que pivot_table ya está creado
pivot_table = df_sa.pivot_table(index=["sex", "fatal"], columns="country", aggfunc="size", fill_value=0)

# Convertir el pivot_table a un DataFrame plano para facilitar la gráfica
flat_df = pivot_table.reset_index()

# Crear el gráfico de barras apiladas
flat_df.set_index(['sex', 'fatal']).plot(kind='bar', stacked=True, ylabel='Count')

# Ajustar el gráfico
plt.title('Count by Country, Sex, and Fatal')
plt.xlabel('Sex and Fatal')
plt.ylabel('Count')
plt.legend(title='Country', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# Mostrar el gráfico
plt.show()


In [ ]:
# Save the cleaned DataFrame to an Excel file
df_sa.to_excel('Cleaned_GSAF5.xlsx', index=False)